Importing Libraries

In [6]:
import pandas as pd
import mysql.connector as sql
import streamlit as st
import plotly.express as px
import os
import json
from streamlit_option_menu import option_menu
from PIL import Image

Data Extraction

Cloning Data from Github

In [9]:
os.environ["GIT_PYTHON_REFRESH"] = "quiet"
import git
from git.repo.base import Repo
Repo.clone_from("https://github.com/PhonePe/pulse.git", "C:/Users/krish/OneDrive/Desktop/phonepe-pulse/data")

GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v -- https://github.com/PhonePe/pulse.git C:/Users/krish/OneDrive/Desktop/phonepe-pulse/data
  stderr: 'fatal: destination path 'C:/Users/krish/OneDrive/Desktop/phonepe-pulse/data' already exists and is not an empty directory.
'

Data transformation

Dataframe of aggregated transactions

In [7]:
path1 = "C:/Users/krish/OneDrive/Desktop/phonepe-pulse/data/pulse/data/aggregated/transaction/country/india/state/"
agg_trans_list = os.listdir(path1)

columns1 = {'State': [], 'Year': [], 'Quarter': [], 'Transaction_type': [], 'Transaction_count': [],
            'Transaction_amount': []}
for state in agg_trans_list:
    cur_state = path1 + state + "/"
    agg_year_list = os.listdir(cur_state)
    
    for year in agg_year_list:
        cur_year = cur_state + year + "/"
        agg_file_list = os.listdir(cur_year)
        
        for file in agg_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            A = json.load(data)
            
            for i in A['data']['transactionData']:
                name = i['name']
                count = i['paymentInstruments'][0]['count']
                amount = i['paymentInstruments'][0]['amount']
                columns1['Transaction_type'].append(name)
                columns1['Transaction_count'].append(count)
                columns1['Transaction_amount'].append(amount)
                columns1['State'].append(state)
                columns1['Year'].append(year)
                columns1['Quarter'].append(int(file.strip('.json')))
                
df_agg_trans = pd.DataFrame(columns1)

In [8]:
df_agg_trans.shape

(4314, 6)

Dataframe of aggregated user

In [9]:
path2 = "C:/Users/krish/OneDrive/Desktop/phonepe-pulse/data/pulse/data/aggregated/user/country/india/state/"

agg_user_list = os.listdir(path2)

columns2 = {'State': [], 'Year': [], 'Quarter': [], 'Brands': [], 'Count': [],
            'Percentage': []}
for state in agg_user_list:
    cur_state = path2 + state + "/"
    agg_year_list = os.listdir(cur_state)
    
    for year in agg_year_list:
        cur_year = cur_state + year + "/"
        agg_file_list = os.listdir(cur_year)

        for file in agg_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            B = json.load(data)
            try:
                for i in B["data"]["usersByDevice"]:
                    brand_name = i["brand"]
                    counts = i["count"]
                    percents = i["percentage"]
                    columns2["Brands"].append(brand_name)
                    columns2["Count"].append(counts)
                    columns2["Percentage"].append(percents)
                    columns2["State"].append(state)
                    columns2["Year"].append(year)
                    columns2["Quarter"].append(int(file.strip('.json')))
            except:
                pass
df_agg_user = pd.DataFrame(columns2)

In [10]:
df_agg_user

,State,Year,Quarter,Brands,Count,Percentage
0,andaman-&-nicobar-islands,2018,1,Xiaomi,1665,0.247033
1,andaman-&-nicobar-islands,2018,1,Samsung,1445,0.214392
2,andaman-&-nicobar-islands,2018,1,Vivo,982,0.145697
3,andaman-&-nicobar-islands,2018,1,Oppo,501,0.074332
4,andaman-&-nicobar-islands,2018,1,OnePlus,332,0.049258
...,...,...,...,...,...,...
6727,west-bengal,2022,1,Lenovo,330017,0.015056
6728,west-bengal,2022,1,Infinix,284678,0.012987
6729,west-bengal,2022,1,Asus,280347,0.012790
6730,west-bengal,2022,1,Apple,277752,0.012671


Dataframe of map transactions

In [11]:
path3 = "C:/Users/krish/OneDrive/Desktop/phonepe-pulse/data/pulse/data/map/transaction/hover/country/india/state/"

map_trans_list = os.listdir(path3)

columns3 = {'State': [], 'Year': [], 'Quarter': [], 'District': [], 'Count': [],
            'Amount': []}

for state in map_trans_list:
    cur_state = path3 + state + "/"
    map_year_list = os.listdir(cur_state)
    
    for year in map_year_list:
        cur_year = cur_state + year + "/"
        map_file_list = os.listdir(cur_year)
        
        for file in map_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            C = json.load(data)
            
            for i in C["data"]["hoverDataList"]:
                district = i["name"]
                count = i["metric"][0]["count"]
                amount = i["metric"][0]["amount"]
                columns3["District"].append(district)
                columns3["Count"].append(count)
                columns3["Amount"].append(amount)
                columns3['State'].append(state)
                columns3['Year'].append(year)
                columns3['Quarter'].append(int(file.strip('.json')))
                
df_map_trans = pd.DataFrame(columns3)

In [12]:
df_map_trans

,State,Year,Quarter,District,Count,Amount
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,442,9.316631e+05
1,andaman-&-nicobar-islands,2018,1,south andaman district,5688,1.256025e+07
2,andaman-&-nicobar-islands,2018,1,nicobars district,528,1.139849e+06
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,825,1.317863e+06
4,andaman-&-nicobar-islands,2018,2,south andaman district,9395,2.394824e+07
...,...,...,...,...,...,...
17559,west-bengal,2023,4,nadia district,36242432,6.879568e+10
17560,west-bengal,2023,4,birbhum district,20708496,3.690499e+10
17561,west-bengal,2023,4,purba medinipur district,40572042,7.981990e+10
17562,west-bengal,2023,4,maldah district,34654950,6.416729e+10


Dataframe of map user

In [13]:
path4 = "C:/Users/krish/OneDrive/Desktop/phonepe-pulse/data/pulse/data/map/user/hover/country/india/state/"

map_user_list = os.listdir(path4)

columns4 = {"State": [], "Year": [], "Quarter": [], "District": [],
            "RegisteredUser": [], "AppOpens": []}

for state in map_user_list:
    cur_state = path4 + state + "/"
    map_year_list = os.listdir(cur_state)
    
    for year in map_year_list:
        cur_year = cur_state + year + "/"
        map_file_list = os.listdir(cur_year)
        
        for file in map_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            D = json.load(data)
            
            for i in D["data"]["hoverData"].items():
                district = i[0]
                registereduser = i[1]["registeredUsers"]
                appOpens = i[1]['appOpens']
                columns4["District"].append(district)
                columns4["RegisteredUser"].append(registereduser)
                columns4["AppOpens"].append(appOpens)
                columns4['State'].append(state)
                columns4['Year'].append(year)
                columns4['Quarter'].append(int(file.strip('.json')))
                
df_map_user = pd.DataFrame(columns4)

In [14]:
df_map_user

,State,Year,Quarter,District,RegisteredUser,AppOpens
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,632,0
1,andaman-&-nicobar-islands,2018,1,south andaman district,5846,0
2,andaman-&-nicobar-islands,2018,1,nicobars district,262,0
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,911,0
4,andaman-&-nicobar-islands,2018,2,south andaman district,8143,0
...,...,...,...,...,...,...
17563,west-bengal,2023,4,nadia district,1603527,67224546
17564,west-bengal,2023,4,birbhum district,1013025,52281841
17565,west-bengal,2023,4,purba medinipur district,1589203,69991821
17566,west-bengal,2023,4,maldah district,1134600,88292475


Dataframe of top transactions

In [15]:
path5 = "C:/Users/krish/OneDrive/Desktop/phonepe-pulse/data/pulse/data/top/transaction/country/india/state/"

top_trans_list = os.listdir(path5)
columns5 = {'State': [], 'Year': [], 'Quarter': [], 'Pincode': [], 'Transaction_count': [],
            'Transaction_amount': []}

for state in top_trans_list:
    cur_state = path5 + state + "/"
    top_year_list = os.listdir(cur_state)
    
    for year in top_year_list:
        cur_year = cur_state + year + "/"
        top_file_list = os.listdir(cur_year)
        
        for file in top_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            E = json.load(data)
            
            for i in E['data']['pincodes']:
                name = i['entityName']
                count = i['metric']['count']
                amount = i['metric']['amount']
                columns5['Pincode'].append(name)
                columns5['Transaction_count'].append(count)
                columns5['Transaction_amount'].append(amount)
                columns5['State'].append(state)
                columns5['Year'].append(year)
                columns5['Quarter'].append(int(file.strip('.json')))
df_top_trans = pd.DataFrame(columns5)
            

In [16]:
df_top_trans

,State,Year,Quarter,Pincode,Transaction_count,Transaction_amount
0,andaman-&-nicobar-islands,2018,1,744101,1622,2.769298e+06
1,andaman-&-nicobar-islands,2018,1,744103,1223,2.238042e+06
2,andaman-&-nicobar-islands,2018,1,744102,969,3.519060e+06
3,andaman-&-nicobar-islands,2018,1,744105,685,1.298561e+06
4,andaman-&-nicobar-islands,2018,1,744104,340,1.039715e+06
...,...,...,...,...,...,...
8562,west-bengal,2023,4,721301,4133775,5.718222e+09
8563,west-bengal,2023,4,700001,3983241,8.039689e+09
8564,west-bengal,2023,4,732125,3950411,6.557345e+09
8565,west-bengal,2023,4,700039,3742452,5.608622e+09


Dataframe of top users

In [17]:
path6 = "C:/Users/krish/OneDrive/Desktop/phonepe-pulse/data/pulse/data/top/user/country/india/state/"
top_user_list = os.listdir(path6)
columns6 = {'State': [], 'Year': [], 'Quarter': [], 'Pincode': [],
            'RegisteredUsers': []}

for state in top_user_list:
    cur_state = path6 + state + "/"
    top_year_list = os.listdir(cur_state)
    
    for year in top_year_list:
        cur_year = cur_state + year + "/"
        top_file_list = os.listdir(cur_year)
        
        for file in top_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            F = json.load(data)
            
            for i in F['data']['pincodes']:
                name = i['name']
                registeredUsers = i['registeredUsers']
                columns6['Pincode'].append(name)
                columns6['RegisteredUsers'].append(registeredUsers)
                columns6['State'].append(state)
                columns6['Year'].append(year)
                columns6['Quarter'].append(int(file.strip('.json')))
df_top_user = pd.DataFrame(columns6)         

In [18]:
df_top_user

,State,Year,Quarter,Pincode,RegisteredUsers
0,andaman-&-nicobar-islands,2018,1,744103,1608
1,andaman-&-nicobar-islands,2018,1,744101,1108
2,andaman-&-nicobar-islands,2018,1,744105,1075
3,andaman-&-nicobar-islands,2018,1,744102,1006
4,andaman-&-nicobar-islands,2018,1,744104,272
...,...,...,...,...,...
8563,west-bengal,2023,4,700015,126663
8564,west-bengal,2023,4,742304,123320
8565,west-bengal,2023,4,721101,123088
8566,west-bengal,2023,4,700150,119283


Converting dataframe to csv files

In [19]:
df_agg_trans.to_csv('agg_trans.csv',index=False)
df_agg_user.to_csv('agg_user.csv',index=False)
df_map_trans.to_csv('map_trans.csv',index=False)
df_map_user.to_csv('map_user.csv',index=False)
df_top_trans.to_csv('top_trans.csv',index=False)
df_top_user.to_csv('top_user.csv',index=False)

In [20]:
pwd

'C:\\Users\\krish\\OneDrive\\Desktop\\phonepe-pulse'

Creating connection with mysql

In [21]:
mydb = sql.connect(host="localhost",
                   user="root",
                   password="mysql@50345034",
                   database= "pulse_data",
                   auth_plugin='mysql_native_password'
                  )
mycursor = mydb.cursor(buffered=True)

Create agg trans table

In [22]:
mycursor.execute("create table agg_trans (State varchar(100), Year int, Quarter int, Transaction_type varchar(100), Transaction_count int, Transaction_amount double)")

for i,row in df_agg_trans.iterrows():
    #here %S means string values 
    sql = "INSERT INTO agg_trans VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    # the connection is not auto committed by default, so we must commit to save our changes
    mydb.commit()

Create agg user table

In [23]:
mycursor.execute("create table agg_user (State varchar(100), Year int, Quarter int, Brands varchar(100), Count int, Percentage double)")

for i,row in df_agg_user.iterrows():
    sql = "INSERT INTO agg_user VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

Create map trans table

In [24]:
mycursor.execute("create table map_trans (State varchar(100), Year int, Quarter int, District varchar(100), Count int, Amount double)")

for i,row in df_map_trans.iterrows():
    sql = "INSERT INTO map_trans VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

Create map user table

In [25]:
mycursor.execute("create table map_user (State varchar(100), Year int, Quarter int, District varchar(100), Registered_user int, App_opens int)")

for i,row in df_map_user.iterrows():
    sql = "INSERT INTO map_user VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

Create top trans table

In [26]:
mycursor.execute("create table top_trans (State varchar(100), Year int, Quarter int, Pincode int, Transaction_count int, Transaction_amount double)")

for i,row in df_top_trans.iterrows():
    sql = "INSERT INTO top_trans VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

Create top user table

In [27]:
mycursor.execute("create table top_user (State varchar(100), Year int, Quarter int, Pincode int, Registered_users int)")

for i,row in df_top_user.iterrows():
    sql = "INSERT INTO top_user VALUES (%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

List of tables

In [28]:
mycursor.execute("show tables")
mycursor.fetchall()

[('agg_trans',),
 ('agg_user',),
 ('map_trans',),
 ('map_user',),
 ('top_trans',),
 ('top_user',)]

In [ ]:
pwd

'c:\\Users\\krish\\OneDrive\\Desktop\\phonepe-pulse'

In [3]:
pip install mysql.connector.python



  Obtaining dependency information for mysql.connector.python from https://files.pythonhosted.org/packages/d9/91/007a0d60fee8db4f7385075dc50bf62d2d359b417b374ec06b06ce6c2d64/mysql_connector_python-8.3.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/15.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.4 MB 140.3 kB/s eta 0:01:50
   ---------------------------------------- 0.1/15.4 MB 182.2 kB/s eta 0:01:25
   ---------------------------------------- 0.1/15.4 MB 312.2 kB/s eta 0:00:50
   -------------------------------

In [5]:
pip install streamlit_option_menu


  Obtaining dependency information for streamlit_option_menu from https://files.pythonhosted.org/packages/d9/e5/a04e14be8b5faf9c20d330f84490b4184d2bec558719c1def1824cc4862d/streamlit_option_menu-0.3.12-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/822.1 kB ? eta -:--:--
   - -------------------------------------- 30.7/822.1 kB ? eta -:--:--
   --- ----------------------------------- 81.9/822.1 kB 907.3 kB/s eta 0:00:01
   ---- ----------------------------------- 92.2/822.1 kB 1.0 MB/s eta 0:00:01
   ---- ----------------------------------- 92.2/822.1 kB 1.0 MB/s eta 0:00:01
   ---------- ----------------------------- 225.3/822.1 kB 1.1 MB/s eta 0:00:01
   ---------- ----------------------------- 225.3/822.1 kB 1.1 MB/s eta 0:00:01
   ----------------- ---------------------- 368.6/822.1 kB 1.2 MB/s eta 0:00:01
   --------------------- ------------------ 440.3/822.1 kB 1.3 MB/s eta 0:00:01
   ----------------------------- ---------- 604.2/822.1 kB 1.5 MB/s eta